# Instruction Finetuning using LoRA

This notebook looks into how to perform instruction finetuning using LoRA PEFT method. The task is to perform Supervised finetuning (SFT) of OpenHathi model on a small Hingligh instruct dataset.